In [ ]:
%pip install tensorflow
%pip install keras

In [28]:
import tensorflow as tf

from keras import datasets, layers, models
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout, Concatenate, concatenate
import matplotlib.pyplot as plt

![inception module diagram](inception_module_diagram.png)

Image from C. Szegedy et al: [Going Deeper with Convolutions](https://arxiv.org/abs/1409.4842)

The below function implements the Inception module with dimensions (pictured above on the right)

In [22]:
# Inception model with dimension Reductions

def inception_module(model, f1_conv1, f2_conv1, f2_conv3, f3_conv1, f3_conv5, f4_pool):
    path1 = Conv2D(filters=f1_conv1, kernel_size=(1, 1), activation='relu', padding='same')(model)

    path2 = Conv2D(filters=f2_conv1, kernel_size=(1, 1), activation='relu', padding='same')(model)
    path2 = Conv2D(filters=f2_conv3, kernel_size=(3, 3), activation='relu', padding='same')(path2)

    path3 = Conv2D(filters=f3_conv1, kernel_size=(1, 1), activation='relu', padding='same')(model)
    path3 = Conv2D(filters=f3_conv5, kernel_size=(5, 5), activation='relu', padding='same')(path3)

    path4 = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(model)
    path4 = Conv2D(filters=f4_pool, kernel_size=(1, 1), activation='relu', padding='same')(path4)

    output_layer = concatenate([path1, path2, path3, path4], axis = -1)

    return output_layer

![GoogleLeNet model table](googlenet_model_table.png)

Image from C. Szegedy et al: [Going Deeper with Convolutions](https://arxiv.org/abs/1409.4842)

Below implements the GoogLeNet Model with parameters shown in the above table

In [25]:
# GoogLeNet Model
input = Input(shape=(224, 224, 3)) # 32x32 input layer because of CIFAR-10 image size

# First Convolutional Layer
x = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same', activation='relu')(input)

# First Max Pooling Layer
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

x = Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1), padding='same', activation='relu')(x)

x = inception_module(model=x, f1_conv1=64, f2_conv1=96, f2_conv3=128, f3_conv1=16, f3_conv5=32, f4_pool=32)
x = inception_module(model=x, f1_conv1=128, f2_conv1=128, f2_conv3=192, f3_conv1=32, f3_conv5=96, f4_pool=64)

x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

x = inception_module(model=x, f1_conv1=192, f2_conv1=96, f2_conv3=208, f3_conv1=16, f3_conv5=48, f4_pool=64)
x = inception_module(model=x, f1_conv1=160, f2_conv1=112, f2_conv3=224, f3_conv1=24, f3_conv5=64, f4_pool=64)
x = inception_module(model=x, f1_conv1=128, f2_conv1=128, f2_conv3=256, f3_conv1=24, f3_conv5=64, f4_pool=64)
x = inception_module(model=x, f1_conv1=112, f2_conv1=144, f2_conv3=288, f3_conv1=32, f3_conv5=64, f4_pool=64)
x = inception_module(model=x, f1_conv1=256, f2_conv1=160, f2_conv3=320, f3_conv1=32, f3_conv5=128, f4_pool=128)

x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

x = inception_module(model=x, f1_conv1=256, f2_conv1=160, f2_conv3=320, f3_conv1=32, f3_conv5=128, f4_pool=128)
x = inception_module(model=x, f1_conv1=384, f2_conv1=192, f2_conv3=384, f3_conv1=48, f3_conv5=128, f4_pool=128)

x = AveragePooling2D(pool_size=(7, 7), strides=(1, 1))(x)

x = Dropout(0.4)(x)
x = Dense(1000, activation='softmax')(x)
  
# GoogLeNet Model
model = Model(input, [x], name='GoogLeNet')

In [29]:
model.summary()

Model: "GoogLeNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_131 (Conv2D)            (None, 112, 112, 64  9472        ['input_7[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_33 (MaxPooling2D  (None, 56, 56, 64)  0           ['conv2d_131[0][0]']             
 )                                                                                        

In [31]:
import os
import math

import numpy as np
from PIL import Image
import pandas as pd 
import matplotlib.pyplot as plt


Implementation based on [Khuyen Le's implementation](https://medium.com/mlearning-ai/implementation-of-googlenet-on-keras-d9873aeed83c)